<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/clones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install reportlab
!pip install pdfkit
!pip install tqdm
!pip install dateparser
!pip install unidecode
!pip install Levenshtein

import pandas as pd
import numpy as np
from datetime import datetime
import locale
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table
import pdfkit
import tabulate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import html
import dateparser
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
from tqdm import tqdm
from unidecode import unidecode
import Levenshtein
from nltk.metrics.distance import jaccard_distance

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
dossier = '/content/drive/MyDrive/Stage/sas/sas/'

# Liste pour stocker les DataFrames
dataframes = []

# Boucle pour créer ma liste de DataFrames

for i in tqdm(range(2011,2023)):
    # Obtenir la liste des fichiers CSV dans le dossier
    chemins_fichiers_sas = dossier+f'a{i}.sas7bdat'
    dataframes.append(pd.read_sas(chemins_fichiers_sas))

df_insee1 = pd.concat(dataframes)
df_insee1 = df_insee1.rename(columns = {'ybirth':'b_annee','mbirth':'b_mois',
                                        'dbirth':'b_jour','ydeath':'d_annee',
                                        'mdeath':'d_mois','ddeath':'d_jour'})

#df_insee1['b_nom'] = df_ins['nom'].str.decode('utf-8', 'ignore')[2:-1]
#df_insee1['b_prenoms'] = df_ins['prenoms'].str.decode('utf-8', 'ignore')[2:-1]
"""
df_insee1['b_annee'] = df_insee1['b_annee'].astype(int)
df_insee1['d_annee'] = df_insee1['d_annee'].astype(int)
df_insee1['b_jour'] = df_insee1['b_jour'].astype(int)
df_insee1['d_jour'] = df_insee1['d_jour'].astype(int)
df_insee1['b_mois'] = df_insee1['b_mois'].astype(int)
df_insee1['d_mois'] = df_insee1['d_mois'].astype(int)
"""

def convert_to_datetime(row):
    try:
        year = int(row['b_annee'])
        month = int(row['b_mois'])
        day = int(row['b_jour'])
        return pd.Timestamp(year=year, month=month, day=day)
    except ValueError:
        return pd.NaT

df_insee1['date_naissance'] = df_insee1.apply(convert_to_datetime, axis=1)

def convert_to_datetime(row):
    try:
        year = int(row['d_annee'])
        month = int(row['d_mois'])
        day = int(row['d_jour'])
        return pd.Timestamp(year=year, month=month, day=day)
    except ValueError:
        return pd.NaT

df_insee1['date_deces'] = df_insee1.apply(convert_to_datetime, axis=1)

df_insee1['Td'] = (df_insee1['date_deces'] - df_insee1['date_naissance']).dt.days
df_insee1['b_dep'] = (df_insee1['codebirth'].str.decode('utf-8')).str.slice(0,2)
#df_insee1['d_pays'] = df_ins['paysbirth'].str.decode('iso-8859-1')
#df_insee1['d_ville'] = df_ins['villebirth'].str.decode('iso-8859-1')
#df_insee1['bcode'] = df_ins['codebirth']
#df_insee1['nbdeath'] = df_ins['nbdeath']
#df_insee1['type'] = df_ins['type']
#df_insee1['reserve'] = df_ins['reserve']
#df_insee1['fichier'] = df_ins['fichier']
#df_insee1['allobs'] = df_ins['allobs']
df_insee1['b_sexe'] = df_insee1['sex']
df_insee1['b_dep']= pd.to_numeric(df_insee1['b_dep'], errors='coerce').astype(float)


insee = df_insee1[['b_annee','b_mois','b_jour','d_annee','d_mois','d_jour','b_sexe','b_dep','Td', 'date_naissance','date_deces']]

100%|██████████| 12/12 [01:21<00:00,  6.78s/it]


In [4]:
"""
#df = pd.read_csv('matching_final.csv')
fichier_clone = df[['id','Td','b_nom','b_prenoms','b_sexe','b_annee', 'b_mois', 'b_jour','b_dep','annee','date_naissance','date_deces']]
fichier_clone['clone'] = 0
fichier_clone['d_annee_clone'] = np.nan
fichier_clone['d_mois_clone'] = np.nan
fichier_clone['d_jour_clone'] = np.nan
fichier_clone['nb_clone'] = 0
"""

<ipython-input-4-befb368ac4b9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fichier_clone['clone'] = 0
<ipython-input-4-befb368ac4b9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fichier_clone['d_annee_clone'] = np.nan
<ipython-input-4-befb368ac4b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [55]:
fichier_clone = pd.read_csv('clone/clone3.csv')
for i in fichier_clone[fichier_clone['nb_clone']==0].index.to_list():

  year = fichier_clone['b_annee'][i]
  month = fichier_clone['b_mois'][i]
  day = fichier_clone['b_jour'][i]

  if type(year)!=pd.Series and pd.notnull(year) and pd.notnull(month) and pd.notnull(day):
      match_insee1 = insee[insee['b_annee'] == year]
      match_insee12 = match_insee1[match_insee1['b_mois']==month]
      match_insee123 = match_insee12[match_insee12['b_jour']==day]
      match_insee2 = match_insee123[match_insee123['b_sexe']==fichier_clone['b_sexe'][i]]
      match_insee3 = match_insee2[match_insee2['b_dep']==fichier_clone['b_dep'][i]]
      id = fichier_clone['id'][i]
      match_insee3['id'] = float(id)
      annee = fichier_clone['annee'][i]
      match_insee3['annee'] = float(annee)


      if fichier_clone['nb_clone'][i]!=0:
        iteration_depart = max(fichier_clone['clone'].to_list())
        match_insee3['clone'] = list(range(int(iteration_depart) + 1, int(match_insee3.shape[0]) + 1 + int(iteration_depart)))
        fichier_clone['nb_clone'][i] = int(match_insee3.shape[0]) + int(len(fichier_clone['clone'].to_list())) - 1
      elif fichier_clone['nb_clone'][i]==0:
        match_insee3['clone'] = list(range(1, match_insee3.shape[0]+1))
        fichier_clone['nb_clone'][i] = match_insee3.shape[0]


      fichier_clone = fichier_clone.append(match_insee3)

<ipython-input-55-b718be0aa885>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_insee3['id'] = float(id)
<ipython-input-55-b718be0aa885>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_insee3['annee'] = float(annee)
<ipython-input-55-b718be0aa885>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [ ]:
# Individus qui n'ont pas été matchés sur les départements
for i in fichier_clone[fichier_clone['nb_clone']==0].index.to_list():

  year = fichier_clone['b_annee'][i]
  month = fichier_clone['b_mois'][i]
  day = fichier_clone['b_jour'][i]

  if type(year)!=pd.Series and pd.notnull(year) and pd.notnull(month) and pd.notnull(day):
      match_insee1 = insee[insee['b_annee'] == year]
      match_insee12 = match_insee1[match_insee1['b_mois']==month]
      match_insee123 = match_insee12[match_insee12['b_jour']==day]
      match_insee2 = match_insee123[match_insee123['b_sexe']==fichier_clone['b_sexe'][i]]
      #match_insee3 = match_insee2[match_insee2['b_dep']==fichier_clone['b_dep'][i]]
      id = fichier_clone['id'][i]
      match_insee2['id'] = float(id)
      annee = fichier_clone['annee'][i]
      match_insee2['annee'] = float(annee)


      if fichier_clone['nb_clone'][i]!=0:
        iteration_depart = max(fichier_clone['clone'].to_list())
        match_insee2['clone'] = list(range(int(iteration_depart) + 1, int(match_insee2.shape[0]) + 1 + int(iteration_depart)))
        fichier_clone['nb_clone'][i] = int(match_insee2.shape[0]) + int(len(fichier_clone['clone'].to_list())) - 1
      elif fichier_clone['nb_clone'][i]==0:
        match_insee2['clone'] = list(range(1, match_insee2.shape[0]+1))
        fichier_clone['nb_clone'][i] = match_insee2.shape[0]


      fichier_clone = fichier_clone.append(match_insee2)

In [56]:
fichier_clone = fichier_clone.sort_values(by = 'id')
fichier_clone['nb_clone'].describe()

count    1025.000000
mean        8.913171
std         9.226071
min         1.000000
25%         2.000000
50%         4.000000
75%        14.000000
max        46.000000
Name: nb_clone, dtype: float64

In [52]:
fichier_clone.to_csv('clone/clone4.csv')

In [58]:
fichier_clone

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,Td,b_nom,b_prenoms,b_sexe,b_annee,b_mois,...,date_naissance,date_deces,clone,d_annee_clone,d_mois_clone,d_jour_clone,nb_clone,d_annee,d_mois,d_jour
0,0.0,0.0,0.0,1.0,31235.0,ABOMES,PAULE MARIE,2.0,1919.0,8.0,...,1919-08-14,2005-02-18,0.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
7,1.0,668379.0,NaN,1.0,29330.0,NaN,NaN,2.0,1919.0,8.0,...,1919-08-14 00:00:00,1999-12-02 00:00:00,3.0,NaN,NaN,NaN,NaN,1999.0,12.0,2.0
6,2.0,690371.0,NaN,1.0,29319.0,NaN,NaN,2.0,1919.0,8.0,...,1919-08-14 00:00:00,1999-11-21 00:00:00,4.0,NaN,NaN,NaN,NaN,1999.0,11.0,21.0
5,3.0,341433.0,NaN,1.0,29072.0,NaN,NaN,2.0,1919.0,8.0,...,1919-08-14 00:00:00,1999-03-19 00:00:00,2.0,NaN,NaN,NaN,NaN,1999.0,3.0,19.0
8,4.0,1920.0,1559.0,1.0,30860.0,NaN,NaN,2.0,1905.0,2.0,...,1905-02-27 00:00:00,1989-08-25 00:00:00,4.0,NaN,NaN,NaN,NaN,1989.0,8.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18329,18222.0,14174.0,353471.0,1186.0,17707.0,NaN,NaN,2.0,1930.0,4.0,...,1930-04-05 00:00:00,1978-09-27 00:00:00,1.0,NaN,NaN,NaN,NaN,1978.0,9.0,27.0
18330,18217.0,983.0,983.0,1186.0,32435.0,DE PLAN DE SIEYES DE VEYNES,CLAUDE ANTOINETTE MARIE,2.0,1930.0,4.0,...,1930-04-05,2019-01-23,0.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
18331,18223.0,984.0,984.0,1187.0,31886.0,DE LA BIGNE,ANNE MARIE,2.0,1911.0,1.0,...,1911-01-11,1998-04-30,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
18333,18225.0,14180.0,235028.0,1187.0,26776.0,NaN,NaN,2.0,1911.0,1.0,...,1911-01-11 00:00:00,1984-05-03 00:00:00,2.0,NaN,NaN,NaN,NaN,1984.0,5.0,3.0


In [35]:
# Corriger le nombre de clones pour chaque individu

for i in fichier_clone[fichier_clone['id']==0].index.to_list():
  # Retrancher le "clone 0 qui correspond à l'individu"
  nb_clone = int(fichier_clone[fichier_clone['id']==fichier_clone['id'][i]].shape[0]) - 1
  fichier_clone['nb_clone'][i] = nb_clone

,id,Td,b_nom,b_prenoms,b_sexe,b_annee,b_mois,b_jour,b_dep,annee,date_naissance,date_deces,clone,d_annee_clone,d_mois_clone,d_jour_clone,nb_clone,d_annee,d_mois,d_jour
0,1.0,31235.0,ABOMES,PAULE MARIE,2.0,1919.0,8.0,14.0,29.0,1994.0,1919-08-14,2005-02-18,0.0,NaN,NaN,NaN,14498.0,NaN,NaN,NaN
31653,1.0,26760.0,NaN,NaN,2.0,1919.0,8.0,14.0,29.0,1994.0,1919-08-14 00:00:00,1992-11-18 00:00:00,1.0,NaN,NaN,NaN,NaN,1992.0,11.0,18.0
341433,1.0,29072.0,NaN,NaN,2.0,1919.0,8.0,14.0,29.0,1994.0,1919-08-14 00:00:00,1999-03-19 00:00:00,2.0,NaN,NaN,NaN,NaN,1999.0,3.0,19.0
668379,1.0,29330.0,NaN,NaN,2.0,1919.0,8.0,14.0,29.0,1994.0,1919-08-14 00:00:00,1999-12-02 00:00:00,3.0,NaN,NaN,NaN,NaN,1999.0,12.0,2.0
690371,1.0,29319.0,NaN,NaN,2.0,1919.0,8.0,14.0,29.0,1994.0,1919-08-14 00:00:00,1999-11-21 00:00:00,4.0,NaN,NaN,NaN,NaN,1999.0,11.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275545,1187.0,32196.0,NaN,NaN,2.0,1911.0,1.0,11.0,56.0,1984.0,1911-01-11 00:00:00,1999-03-06 00:00:00,4268.0,NaN,NaN,NaN,NaN,1999.0,3.0,6.0
275546,1187.0,32196.0,NaN,NaN,2.0,1911.0,1.0,11.0,56.0,1984.0,1911-01-11 00:00:00,1999-03-06 00:00:00,4269.0,NaN,NaN,NaN,NaN,1999.0,3.0,6.0
275546,1187.0,32196.0,NaN,NaN,2.0,1911.0,1.0,11.0,56.0,1984.0,1911-01-11 00:00:00,1999-03-06 00:00:00,38692.0,NaN,NaN,NaN,NaN,1999.0,3.0,6.0
236764,1187.0,26452.0,NaN,NaN,2.0,1911.0,1.0,11.0,56.0,1984.0,1911-01-11 00:00:00,1983-06-14 00:00:00,1.0,NaN,NaN,NaN,56856.0,1983.0,6.0,14.0
